# AIPI 590 - XAI | Interpretable AI
### Understand a dataset and develop interpretable models to make predictions
### Mariam Khan

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mkhan13/XAI/blob/interpretable_ml_assignment/khan_interpretable_ml.ipynb)